# Подготовка данных

In [44]:
import pandas as pd
import numpy as np
import pathlib
import sys

ROOT = pathlib.Path().resolve().parent

sys.path.insert(0, str(ROOT))

df = pd.read_csv(ROOT / 'data/dataset.csv', sep='\t')
df

,category_ru,sing_ru,date,month,day,weekday,weekday_ru,text_ru
0,общее,овен,2013-01-12 00:01:00,1,12,5,Суббота,"Вы не из тех, кто перестраховывается, и даже е..."
1,общее,овен,2013-01-12 00:02:00,1,12,5,Суббота,"Не существует такого понятия, как ""что-то даро..."
2,общее,овен,2013-01-12 00:03:00,1,12,5,Суббота,Поскольку новолуние приходится на одну из наиб...
3,общее,овен,2013-01-12 00:04:00,1,12,5,Суббота,"Сегодня вы услышите что-то потрясающее, но мож..."
4,общее,овен,2013-01-12 00:05:00,1,12,5,Суббота,"Друг или коллега, которого вы давно не видели,..."
...,...,...,...,...,...,...,...,...
34900,др,рыбы,2021-01-12 00:06:00,1,12,1,Вторник,"Отпразднуйте свой день рождения со вкусом, гот..."
34901,др,рыбы,2021-01-13 00:06:00,1,13,2,Среда,"Представьте свою жизнь так, как если бы она бы..."
34902,др,рыбы,2021-01-14 00:06:00,1,14,3,Четверг,"Веселье, игривость и юмор легко проявить в это..."
34903,др,рыбы,2021-01-15 00:06:00,1,15,4,Пятница,"Ваш день рождения - это новый старт, когда вы ..."


## Transformers

Есть пример похожей работы на transformers [ссылка](https://huggingface.co/shahp7575/gpt2-horoscopes)


**Для работы модели текстовой генерации необходимо несколько вещей:**

1. Предобученная модель на массиве текстов
2. Адекватный векторный промт - то, что модель будет продолжать

В качестве фичеров для такой маски мы предлагаем использовать следующий подход:

`<Категория> {Категория} <День недели>`


In [45]:
def make_prompt(weekday, category, sign):
    return f"<день недели> {weekday} <категория> {category} <знак> {sign} "

def make_full_prompt(weekday, category, sign, text):
    return make_prompt(weekday, category, sign) + f"Гороскоп: {text}"

In [46]:
make_full_prompt('Пятница', "Общее", "Рыбы", "")

'<день недели> Пятница <категория> Общее <знак> Рыбы Гороскоп: '

In [47]:
df['features'] = df[['weekday_ru','category_ru', 'sing_ru']].apply(lambda x: make_prompt(x[0], x[1], x[2]), axis=1)
df

,category_ru,sing_ru,date,month,day,weekday,weekday_ru,text_ru,features
0,общее,овен,2013-01-12 00:01:00,1,12,5,Суббота,"Вы не из тех, кто перестраховывается, и даже е...",<день недели> Суббота <категория> общее <знак>...
1,общее,овен,2013-01-12 00:02:00,1,12,5,Суббота,"Не существует такого понятия, как ""что-то даро...",<день недели> Суббота <категория> общее <знак>...
2,общее,овен,2013-01-12 00:03:00,1,12,5,Суббота,Поскольку новолуние приходится на одну из наиб...,<день недели> Суббота <категория> общее <знак>...
3,общее,овен,2013-01-12 00:04:00,1,12,5,Суббота,"Сегодня вы услышите что-то потрясающее, но мож...",<день недели> Суббота <категория> общее <знак>...
4,общее,овен,2013-01-12 00:05:00,1,12,5,Суббота,"Друг или коллега, которого вы давно не видели,...",<день недели> Суббота <категория> общее <знак>...
...,...,...,...,...,...,...,...,...,...
34900,др,рыбы,2021-01-12 00:06:00,1,12,1,Вторник,"Отпразднуйте свой день рождения со вкусом, гот...",<день недели> Вторник <категория> др <знак> рыбы
34901,др,рыбы,2021-01-13 00:06:00,1,13,2,Среда,"Представьте свою жизнь так, как если бы она бы...",<день недели> Среда <категория> др <знак> рыбы
34902,др,рыбы,2021-01-14 00:06:00,1,14,3,Четверг,"Веселье, игривость и юмор легко проявить в это...",<день недели> Четверг <категория> др <знак> рыбы
34903,др,рыбы,2021-01-15 00:06:00,1,15,4,Пятница,"Ваш день рождения - это новый старт, когда вы ...",<день недели> Пятница <категория> др <знак> рыбы


In [48]:
df['full'] = df[['weekday_ru', 'category_ru', 'sing_ru', 'text_ru']].apply(lambda x: make_full_prompt(x[0], x[1], x[2], x[3]), axis=1)
df

,category_ru,sing_ru,date,month,day,weekday,weekday_ru,text_ru,features,full
0,общее,овен,2013-01-12 00:01:00,1,12,5,Суббота,"Вы не из тех, кто перестраховывается, и даже е...",<день недели> Суббота <категория> общее <знак>...,<день недели> Суббота <категория> общее <знак>...
1,общее,овен,2013-01-12 00:02:00,1,12,5,Суббота,"Не существует такого понятия, как ""что-то даро...",<день недели> Суббота <категория> общее <знак>...,<день недели> Суббота <категория> общее <знак>...
2,общее,овен,2013-01-12 00:03:00,1,12,5,Суббота,Поскольку новолуние приходится на одну из наиб...,<день недели> Суббота <категория> общее <знак>...,<день недели> Суббота <категория> общее <знак>...
3,общее,овен,2013-01-12 00:04:00,1,12,5,Суббота,"Сегодня вы услышите что-то потрясающее, но мож...",<день недели> Суббота <категория> общее <знак>...,<день недели> Суббота <категория> общее <знак>...
4,общее,овен,2013-01-12 00:05:00,1,12,5,Суббота,"Друг или коллега, которого вы давно не видели,...",<день недели> Суббота <категория> общее <знак>...,<день недели> Суббота <категория> общее <знак>...
...,...,...,...,...,...,...,...,...,...,...
34900,др,рыбы,2021-01-12 00:06:00,1,12,1,Вторник,"Отпразднуйте свой день рождения со вкусом, гот...",<день недели> Вторник <категория> др <знак> рыбы,<день недели> Вторник <категория> др <знак> ры...
34901,др,рыбы,2021-01-13 00:06:00,1,13,2,Среда,"Представьте свою жизнь так, как если бы она бы...",<день недели> Среда <категория> др <знак> рыбы,<день недели> Среда <категория> др <знак> рыбы...
34902,др,рыбы,2021-01-14 00:06:00,1,14,3,Четверг,"Веселье, игривость и юмор легко проявить в это...",<день недели> Четверг <категория> др <знак> рыбы,<день недели> Четверг <категория> др <знак> ры...
34903,др,рыбы,2021-01-15 00:06:00,1,15,4,Пятница,"Ваш день рождения - это новый старт, когда вы ...",<день недели> Пятница <категория> др <знак> рыбы,<день недели> Пятница <категория> др <знак> ры...


In [49]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(df, random_state=42, stratify=df[['sing_ru', 'category_ru']] , test_size=0.2)

In [51]:
train_texts = X_train['full']
valid_texts = X_test['full']

In [52]:
with open('train.txt', 'w') as f:
    for t in train_texts:
        f.write(t + '\n')

In [53]:
with open('valid.txt', 'w') as f:
    for t in valid_texts:
        f.write(t + '\n')